In [ ]:
#!pip install wget
#!pip install -U chromedriver_binary

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import urllib.request 
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import uniform
import re
import os
import pandas as pd
import wget
from datetime import datetime

# auxiliary functions modified by Luis.
import scrape_functions_rba as kzd

def wait_between(a, b):
	arb = uniform(a, b) 
	sleep(arb)

/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


#### FIREFOX SETUP

In [ ]:
from selenium.webdriver.firefox.service import Service

#store_path = "/home/renatovassallo/Omega/BSE/Term2/Textmining/term_paper/Data/statements_peru/"
store_path = "/Users/MargheritaP/Notebooks/Text_mining/Term_paper/"

profile = webdriver.FirefoxProfile()
os.makedirs(store_path, exist_ok=True)
#profile.set_preference("browser.download.dir", store_path)
#profile.set_preference("browser.download.folderList", 2)
#profile.set_preference("browser.download.manager.showWhenStarting", False)
#profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf")
#profile.set_preference("pdfjs.disabled", True)

option = webdriver.FirefoxOptions()
driverService = Service('browser/geckodriver')
driver = webdriver.Firefox(service=driverService, firefox_profile=profile, options=option)


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: firefox_profile has been deprecated, please use an Options object
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  app.launch_new_instance()


#### CHROME SETUP

In [ ]:
#!pip install undetected-chromedriver==2.1.1
#!pip install undetected-chromedriver==1.4

from selenium.webdriver.chrome.service import Service
import undetected_chromedriver as uc

#import chromedriver_binary
#profile = webdriver.Chrome()
#driver = ChromeDriver();
#option = webdriver.ChromeOptions()
driverService = Service('browser/googledriver')

driver = webdriver.Chrome()     #, service=driverService  profile=profile, options=option)

#driver = uc.Chrome()  # This version of ChromeDriver only supports Chrome version 110 - Current browser version is 99.0.4844

SyntaxError: invalid character in identifier (<ipython-input-22-cab7de7bd0e0>, line 13)

#### Getting links from site

In [ ]:
# Get the href links from html using Selenium + BeauifulSoup
main = 'https://www.rba.gov.au'#'https://www.rba.gov.au/'
driver.get(main) # +'/monetary-policy/int-rate-decisions/2018/')
wait_between(2,3)

#soup = BeautifulSoup(driver.page_source, 'html')
#driver.quit()
#soup.find_all('a', href=True)
#soup

In [ ]:
# Collect element of links to statements

years = range(2008,2023) #NB: Prior to December 2007, media releases were issued only when the cash rate target was changed.) 
urls = []

for i in range(len(years)):
    
    year = years[i]
    
    url = f"https://www.rba.gov.au/monetary-policy/int-rate-decisions/{year}"  # {year} placeholder replaced by the current year
    
    driver.get(url)                                   # open link
    soup = BeautifulSoup(driver.page_source, 'html')  # get html from site
    
    text = str(soup)                                   # turn soup to text
    text2 = text.split('<ul class="list-articles"')[1] # split in half and keep all after this('<h1 class="page-title">Monetary Policy Decisions')[1]
    text3 = text2.split('<h2 class="title">Related Information</h2>')[0] # split in half and keep all before this
    
    soup_clean = BeautifulSoup(text3,"html")           # turn central text back to html soup
               
    for link in soup_clean.find_all('a', href=True):   # extract the links and add them to the list (format: # <a href="/media-releases/2022/mr-22-41.html">6 December 2022</a> <a href="/media-releases/2022/mr-22-36.html">1 November 2022</a>)
        urls.append(link['href'])
                            
    wait_between(5,12)                                 # wait before repeating for the next year 

urls    

['/media-releases/2008/mr-08-27.html',
 '/media-releases/2008/mr-08-25.html',
 '/media-releases/2008/mr-08-20.html',
 '/media-releases/2008/mr-08-14.html',
 '/media-releases/2008/mr-08-12.html',
 '/media-releases/2008/mr-08-10.html',
 '/media-releases/2008/mr-08-09.html',
 '/media-releases/2008/mr-08-07.html',
 '/media-releases/2008/mr-08-04.html',
 '/media-releases/2008/mr-08-03.html',
 '/media-releases/2008/mr-08-02.html',
 '/media-releases/2009/mr-09-28.html',
 '/media-releases/2009/mr-09-25.html',
 '/media-releases/2009/mr-09-23.html',
 '/media-releases/2009/mr-09-19.html',
 '/media-releases/2009/mr-09-16.html',
 '/media-releases/2009/mr-09-15.html',
 '/media-releases/2009/mr-09-12.html',
 '/media-releases/2009/mr-09-08.html',
 '/media-releases/2009/mr-09-06.html',
 '/media-releases/2009/mr-09-05.html',
 '/media-releases/2009/mr-09-01.html',
 '/media-releases/2010/mr-10-30.html',
 '/media-releases/2010/mr-10-26.html',
 '/media-releases/2010/mr-10-23.html',
 '/media-releases/2010/mr

In [ ]:
print('Number of links to statements extracted: ' + str(len(urls)))

Number of links to statements extracted: 166


In [ ]:
# save to drive just in case
with open('statement_urls.txt', 'w') as f:
    for item in urls:
        f.write("%s\n" % item)

In [ ]:
# OPTINAL: read back in
with open('statement_urls.txt', 'r') as f:
    saved_urls = [str(line.strip()) for line in f]
saved_urls

['/media-releases/2008/mr-08-27.html',
 '/media-releases/2008/mr-08-25.html',
 '/media-releases/2008/mr-08-20.html',
 '/media-releases/2008/mr-08-14.html',
 '/media-releases/2008/mr-08-12.html',
 '/media-releases/2008/mr-08-10.html',
 '/media-releases/2008/mr-08-09.html',
 '/media-releases/2008/mr-08-07.html',
 '/media-releases/2008/mr-08-04.html',
 '/media-releases/2008/mr-08-03.html',
 '/media-releases/2008/mr-08-02.html',
 '/media-releases/2009/mr-09-28.html',
 '/media-releases/2009/mr-09-25.html',
 '/media-releases/2009/mr-09-23.html',
 '/media-releases/2009/mr-09-19.html',
 '/media-releases/2009/mr-09-16.html',
 '/media-releases/2009/mr-09-15.html',
 '/media-releases/2009/mr-09-12.html',
 '/media-releases/2009/mr-09-08.html',
 '/media-releases/2009/mr-09-06.html',
 '/media-releases/2009/mr-09-05.html',
 '/media-releases/2009/mr-09-01.html',
 '/media-releases/2010/mr-10-30.html',
 '/media-releases/2010/mr-10-26.html',
 '/media-releases/2010/mr-10-23.html',
 '/media-releases/2010/mr

In [ ]:
# Collect statements from each link

statements_list=[] 

for i in range(len(saved_urls)):       #replace saved_urls with urls if not storing locally
    driver.get(main + saved_urls[i])
    wait_between(6,12)
    
    satements = driver.find_elements('xpath', '//*[@id="content"]' )
    
    for matched_element in satements:
        text = matched_element.text
        statements_list.append(text)
        #print(text)

In [ ]:
## How many statements did we scrape?
print('Number of statements extracted: ' + str(len(statements_list)))

print('An example statement: ')
statements_list[5]

Number of statements extracted: 166
An example statement: 


"Media Release\nStatement by Glenn Stevens, Governor: Monetary Policy\nNumber\n2008-10\nDate\n1 July 2008\nAt its meeting today, the Board decided to leave the cash rate unchanged at 7.25 per cent.\nInflation in Australia has been high over the past year in an environment of limited spare capacity and earlier strong growth in demand. In these circumstances, the Board has been seeking to restrain demand in order to reduce inflation over time.\nAs a result of earlier decisions by the Board, additional rises in market interest rates and tougher credit standards for some borrowers, there has been a substantial tightening in financial conditions since the middle of last year. Conditions in international financial markets remain difficult, with credit concerns resurfacing in the past month.\nThe evidence is that the tightening in financial conditions, in conjunction with other factors including rising fuel costs, is working to restrain demand. Indicators of household spending have recorded s

In [ ]:
# Extract dates that appear in the format: "Date 3 December 2019"
#pattern = r'(?<=Date\s).*(?=\s20\d{2})'#pattern = r'(?<=Date\s).*(\d{1,2})\s\b[A-Z][a-z]*\s\d{4}\b)'

pattern = r'(?<=Date\s).*\d{4}'
dates = [re.findall(pattern, s)[0] if re.findall(pattern, s) else None for s in statements_list]

# Combine into a dataframe
df_australia = pd.DataFrame({'extracted_dates': dates, 'Statements': statements_list})
print(df_australia)

      extracted_dates                                         Statements
0     2 December 2008  Media Release\nStatement by Glenn Stevens, Gov...
1     4 November 2008  Media Release\nStatement by Glenn Stevens, Gov...
2      7 October 2008  Media Release\nStatement by Glenn Stevens, Gov...
3    2 September 2008  Media Release\nStatement by Glenn Stevens, Gov...
4       5 August 2008  Media Release\nStatement by Glenn Stevens, Gov...
..                ...                                                ...
161       7 June 2022  Media Release\nStatement by Philip Lowe, Gover...
162        3 May 2022  Media Release\nStatement by Philip Lowe, Gover...
163      5 April 2022  Media Release\nStatement by Philip Lowe, Gover...
164      1 March 2022  Media Release\nStatement by Philip Lowe, Gover...
165   1 February 2022  Media Release\nStatement by Philip Lowe, Gover...

[166 rows x 2 columns]


In [ ]:
df_australia['extracted_dates'] = pd.to_datetime(df_australia['extracted_dates'])
df_australia

,extracted_dates,Statements
0,2008-12-02,"Media Release\nStatement by Glenn Stevens, Gov..."
1,2008-11-04,"Media Release\nStatement by Glenn Stevens, Gov..."
2,2008-10-07,"Media Release\nStatement by Glenn Stevens, Gov..."
3,2008-09-02,"Media Release\nStatement by Glenn Stevens, Gov..."
4,2008-08-05,"Media Release\nStatement by Glenn Stevens, Gov..."
...,...,...
161,2022-06-07,"Media Release\nStatement by Philip Lowe, Gover..."
162,2022-05-03,"Media Release\nStatement by Philip Lowe, Gover..."
163,2022-04-05,"Media Release\nStatement by Philip Lowe, Gover..."
164,2022-03-01,"Media Release\nStatement by Philip Lowe, Gover..."
